In [7]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from pprint import pprint
import zipcodes

# Census & gmaps API Keys
from config import (census_api_key, gkey)
c = Census(census_api_key, year=2017)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [8]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_housing = c.acs5.get(("NAME", "B19013_001E", "B25058_001E", "B25064_001E", "B25077_001E", "B25088_002E"), {'for': 'state:*'}) #'for': 'zip code tabulation area:*' ) 

# Convert to DataFrame
census_housing = pd.DataFrame(census_housing)

# Column Reordering
census_housing = census_housing.rename(columns={
                                      "B19013_001E": "Household Income",
                                      "B25058_001E": "Median Contract Rent",
                                      "B25064_001E": "Median Gross Rent",
                                      "B25077_001E": "Median Home Value",
                                      "B25088_002E": "Median Monthly Owner Costs",
                                      "NAME": "Name","state": "State"})  #"zip code tabulation area": "Zipcode"


# Final DataFrame
census_housing = census_housing[["State", "Name", "Household Income", "Median Contract Rent", "Median Gross Rent", "Median Home Value",
                       "Median Monthly Owner Costs"]]

census_housing.head()

,State,Name,Household Income,Median Contract Rent,Median Gross Rent,Median Home Value,Median Monthly Owner Costs
0,72,Puerto Rico,19775.0,362.0,464.0,115300.0,888.0
1,01,Alabama,46472.0,544.0,747.0,132100.0,1149.0
2,02,Alaska,76114.0,1091.0,1200.0,261900.0,1862.0
3,04,Arizona,53510.0,810.0,972.0,193200.0,1354.0
4,05,Arkansas,43813.0,527.0,709.0,118500.0,1039.0


In [9]:
# Read in the csv containing state happiness rates
happiness = pd.read_csv("./happiness.csv")
happiness.head()

,State,Happiness Rating
0,South Dakota,64.1
1,Vermont,64.1
2,Hawaii,63.4
3,Minnesota,63.1
4,North Dakota,63.1


In [10]:
# Pull commuting data (counts and minutes) from Census API by zip
commute_byst= c.acs5.get(("B08136_003E", "B08136_004E", "B08136_007E","B08136_011E", "B08136_012E","B08301_001E",
                           "B08301_003E","B08301_004E", "B08301_010E","B08301_019E","B08101_041E","NAME"), {'for': 'state:*'})

# Convert to DataFrame
commute_byst_pd = pd.DataFrame(commute_byst)
                                
commute_byst_pd = commute_byst_pd.rename(columns={"B08136_003E":"ct_solo_auto", "B08136_004E":"ct_carpool", "B08136_007E":"ct_pub_trans",
                                                    "B08136_011E":"ct_walked", "B08136_012E":"ct_other","B08301_001E":"t_total","B08301_003E":"t_solo_auto","B08301_004E":"t_carpool",
                                                    "B08301_010E":"t_public_transport","B08301_019E":"t_walked","B08101_041E":"t_other","NAME": "Name"})
commute_byst_pd

,t_other,ct_solo_auto,ct_carpool,ct_pub_trans,ct_walked,ct_other,t_total,t_solo_auto,t_carpool,t_public_transport,t_walked,Name,state
0,19331.0,24161705.0,2761480.0,1057085.0,393255.0,502590.0,1009568.0,828868.0,85272.0,21692.0,32317.0,Puerto Rico,72
1,20531.0,42207585.0,4900280.0,324595.0,241960.0,639890.0,2030100.0,1743217.0,173758.0,7433.0,22659.0,Alabama,01
2,18829.0,4423270.0,871085.0,219085.0,242730.0,733835.0,360701.0,247111.0,44744.0,5453.0,28714.0,Alaska,02
3,76546.0,54323095.0,8771110.0,2832275.0,694310.0,2286730.0,2916850.0,2234990.0,317969.0,58267.0,57699.0,Arizona,04
4,15329.0,22402830.0,3183235.0,159640.0,220485.0,396940.0,1258851.0,1044010.0,132343.0,4663.0,21933.0,Arkansas,05
5,450486.0,355836415.0,58162215.0,45997080.0,6155215.0,12811450.0,17589758.0,12950487.0,1830958.0,909679.0,470101.0,California,06
6,61673.0,50513645.0,6917815.0,3862640.0,922805.0,1816775.0,2742151.0,2062986.0,249838.0,86517.0,78198.0,Colorado,08
7,19988.0,34010705.0,3581590.0,5233535.0,597800.0,570215.0,1773999.0,1387093.0,143925.0,86778.0,51059.0,Connecticut,09
8,22117.0,3400055.0,551085.0,4748585.0,817825.0,529935.0,354658.0,120608.0,19158.0,125641.0,46967.0,District of Columbia,11
9,4990.0,8957090.0,904065.0,597485.0,106125.0,128485.0,435351.0,354471.0,34822.0,12198.0,9244.0,Delaware,10
